In [1]:
import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp
import ipynb.fs.full.utils as utils
from ipynb.fs.full.features import features_independientes_precio, features_dependientes_precio, features_de_csvs

df_train = pd.read_csv('./data/train.csv')
# Para usarse con el submit a Kaggle
df_eval = pd.read_csv('./data/test.csv')

df_train, df_eval = features_de_csvs(df_train, df_eval)

# Randoms solo para asegurarse que los features esten bien hechos
# df_train = df_train.sample(frac=1).reset_index(drop=True)
# df_train = utils.dolarizar_df(df_train)
# df_train = df_train.sample(frac=1).reset_index(drop=True)

df_train, df_test = utils.dividir_df_testeo(df_train, test_size=0.15)

# df_train = df_train.sample(frac=1).reset_index(drop=True)
# df_test = utils.pesificar_df(df_test)
# df_train = df_train.sample(frac=1).reset_index(drop=True)


ModuleNotFoundError: No module named 'ipynb.fs.full.utils'

In [2]:
def llenar_nulls(df):
    df['metrostotales'] = df['metrostotales'].fillna(df['metroscubiertos'])
    df['metroscubiertos'] = df['metrostotales'].fillna(df['metrostotales'])
    
    df['habitaciones'] = df['habitaciones'].fillna(df['habitaciones'].mean())
    df['garages'] = df['garages'].fillna(df['garages'].mean())
    df['banos'] = df['banos'].fillna(df['banos'].mean())
    
    df['fecha'] = pd.to_datetime(df['fecha']).astype(int)

df_test_f = features_independientes_precio(df_test)
df_test_f = features_dependientes_precio(df_test_f, df_train)

df_train_f = features_independientes_precio(df_train)
df_train_f = features_dependientes_precio(df_train_f, df_train)

llenar_nulls(df_train_f)
llenar_nulls(df_test_f)

In [6]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
    
def normalizar_df(df, features):
    min_max = MinMaxScaler()
    df[features] = pd.DataFrame(min_max.fit_transform(df[features]), columns=features)
    return df
    


features = ['habitaciones', 'garages', 'banos', 'fecha',
       'metroscubiertos', 'metrostotales', 'provincia',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']

features_test = ['similares_count',
       'porcentaje_metros', 'diferencia_metros', 'intervalo_metros_totales',
       'intervalo_metros_cubiertos', 'escomercial',
       'promedio_metros_tipo_propiedad', 'promedio_metros_cub_tipo_propiedad',
       'tipo_propiedad_compartida', 'prop_frecuente', 'zona', 'top_provincia',
       'es_ciudad_centrica', 'promedio_metros_totales_provincia',
       'promedio_metros_cubiertos_provincia', 'anio', 'mes', 'dia',
       'trimestre', 'escualas_centros_cercanos', 'delincuencia', 'turismo',
       'es_antigua', 'cantidad_inquilinos', 'metros_totales_normalizados',
       'metros_cubiertos_normalizados', 'promedio_precio_provincia',
       'promedio_precio_ciudad', 'promedio_precio_ciudad_gen',
       'varianza_precio_ciudad', 'count_ciudad', 'promedio_id_zona',
       'promedio_id_zona_gen', 'varianza_id_zona', 'count_id_zona',
       'promedio_precio_tipo_propiedad',
       'promedio_precio_tipo_propiedad_ciudad',
       'promedio_precio_tipo_propiedad_ciudad_gen', 'count_tipo_propiedad',
       'count_tipo_propiedad_ciudad', 'promedio_por_mes', 'varianza_por_mes',
       'promedio_precio_habitaciones',
       'promedio_precio_habitaciones_banos_garages',
       'promedio_precio_banos_garages', 'promedio_precio_hbg_tipo_propiedad',
       'promedio_precio_hbg_tipo_propiedad_provincia',
       'promedio_precio_booleanos', 'puntaje']

features_cat = ['provincia', 'tipodepropiedad', 'intervalo_metros_totales', 'intervalo_metros_cubiertos',
               'zona', 'escualas_centros_cercanos']

features += features_test + features_cat

df_train_g = utils.filtrar_features(df_train_f, features, 'precio')

features_a_normalizar = [f for f in features if f not in features_cat + ['precio']]

df_train_n = pd.get_dummies(df_train_g, columns=features_cat)
# df_train_n = normalizar_df(df_train_n, features_a_normalizar)

x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_n, 'precio', test_size=0.15)

model = Sequential()
model.add(BatchNormalization())
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=1, activation='linear'))

model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'], validation_split=0.2)

model.fit(x_train.values, y_train.values, epochs=15)

y_pred_test = model.predict(x_test)

keras_mae_test = utils.MAE(y_test, y_pred_test)
print(f"MAE Keras (test): {keras_mae_test:.5f}")


Epoch 1/15
173400/173400 [==============================] - 18s 106us/step - loss: 901256.6454 - accuracy: 0.0000e+00
Epoch 2/15
173400/173400 [==============================] - 18s 104us/step - loss: 699336.7062 - accuracy: 5.7670e-06
Epoch 3/15
173400/173400 [==============================] - 18s 105us/step - loss: 683115.0745 - accuracy: 0.0000e+00
Epoch 4/15
173400/173400 [==============================] - 18s 106us/step - loss: 675610.0354 - accuracy: 5.7670e-06
Epoch 5/15
173400/173400 [==============================] - 18s 105us/step - loss: 672480.3987 - accuracy: 0.0000e+00
Epoch 6/15
173400/173400 [==============================] - 18s 105us/step - loss: 669083.8171 - accuracy: 0.0000e+00
Epoch 7/15
173400/173400 [==============================] - 18s 106us/step - loss: 664432.3206 - accuracy: 0.0000e+00
Epoch 8/15
173400/173400 [==============================] - 20s 113us/step - loss: 664941.4515 - accuracy: 0.0000e+00
Epoch 9/15
173400/173400 [==============================

In [33]:

import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp
import ipynb.fs.full.utils as utils
from ipynb.fs.full.features import features_independientes_precio, features_dependientes_precio, features_de_csvs

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
    
def normalizar_df(df, features):
    min_max = MinMaxScaler()
    df[features] = pd.DataFrame(min_max.fit_transform(df[features]), columns=features)
    return df

def llenar_nulls(df):
    df['metrostotales'] = df['metrostotales'].fillna(df['metroscubiertos'])
    df['metroscubiertos'] = df['metrostotales'].fillna(df['metrostotales'])
    
    df['habitaciones'] = df['habitaciones'].fillna(df['habitaciones'].mean())
    df['garages'] = df['garages'].fillna(df['garages'].mean())
    df['banos'] = df['banos'].fillna(df['banos'].mean())
    
    df['fecha'] = pd.to_datetime(df['fecha']).astype(int)

    
df_train = pd.read_csv('./data/train.csv')

# Para usarse con el submit a Kaggle
df_test = pd.read_csv('./data/test.csv')

df_train, df_test = features_de_csvs(df_train, df_test)



# df_train = utils.dolarizar_df(df_train)

df_test_f = features_independientes_precio(df_test)
df_test_f = features_dependientes_precio(df_test_f, df_train)

df_train_f = features_independientes_precio(df_train)
df_train_f = features_dependientes_precio(df_train_f, df_train)

llenar_nulls(df_train_f)
llenar_nulls(df_test_f)


features = ['habitaciones', 'garages', 'banos', 'fecha',
       'metroscubiertos', 'metrostotales', 'provincia',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']

features_test = ['similares_count',
       'porcentaje_metros', 'diferencia_metros', 'intervalo_metros_totales',
       'intervalo_metros_cubiertos', 'escomercial',
       'promedio_metros_tipo_propiedad', 'promedio_metros_cub_tipo_propiedad',
       'tipo_propiedad_compartida', 'prop_frecuente', 'zona', 'top_provincia',
       'es_ciudad_centrica', 'promedio_metros_totales_provincia',
       'promedio_metros_cubiertos_provincia', 'anio', 'mes', 'dia',
       'trimestre', 'escualas_centros_cercanos', 'delincuencia', 'turismo',
       'es_antigua', 'cantidad_inquilinos', 'metros_totales_normalizados',
       'metros_cubiertos_normalizados', 'promedio_precio_provincia',
       'promedio_precio_ciudad', 'promedio_precio_ciudad_gen',
       'varianza_precio_ciudad', 'count_ciudad', 'promedio_id_zona',
       'promedio_id_zona_gen', 'varianza_id_zona', 'count_id_zona',
       'promedio_precio_tipo_propiedad',
       'promedio_precio_tipo_propiedad_ciudad',
       'promedio_precio_tipo_propiedad_ciudad_gen', 'count_tipo_propiedad',
       'count_tipo_propiedad_ciudad', 'promedio_por_mes', 'varianza_por_mes',
       'promedio_precio_habitaciones',
       'promedio_precio_habitaciones_banos_garages',
       'promedio_precio_banos_garages', 'promedio_precio_hbg_tipo_propiedad',
       'promedio_precio_hbg_tipo_propiedad_provincia',
       'promedio_precio_booleanos', 'puntaje']

features_cat = ['provincia', 'tipodepropiedad', 'intervalo_metros_totales', 'intervalo_metros_cubiertos',
               'zona', 'escualas_centros_cercanos']

features += features_test + features_cat


features_a_normalizar = [f for f in features if f not in features_cat + ['precio']]

df_full = pd.concat([df_test_f, df_train_f.drop('precio', axis=1)], ignore_index=True)
df_full_g = utils.filtrar_features(df_full, features, 'precio')

df_full_ohe = pd.get_dummies(df_full_g, columns=features_cat)

df_train_n = df_full_ohe.iloc[len(df_test_f.values):len(df_full_ohe.values)].reset_index()
df_train_n['precio'] = df_train_f['precio']
df_test_n = df_full_ohe.iloc[0:len(df_test_f.values)].reset_index()

x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_n, 'precio', test_size=0.05)

model = Sequential()
model.add(BatchNormalization())
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=1, activation='linear'))

model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'], validation_split=0.1)

model.fit(x_train.values, y_train.values, epochs=15)

y_pred_train = model.predict(x_train)

keras_mae_train = utils.MAE(y_train, y_pred_train)
print(f"MAE Keras (train): {keras_mae_train:.5f}")

Epoch 1/15
228000/228000 [==============================] - 25s 111us/step - loss: 863704.0013 - accuracy: 4.3860e-06
Epoch 2/15
228000/228000 [==============================] - 25s 109us/step - loss: 699117.8261 - accuracy: 0.0000e+00
Epoch 3/15
228000/228000 [==============================] - 25s 110us/step - loss: 683363.8877 - accuracy: 0.0000e+00
Epoch 4/15
228000/228000 [==============================] - 25s 110us/step - loss: 678821.8244 - accuracy: 0.0000e+00
Epoch 5/15
228000/228000 [==============================] - 26s 114us/step - loss: 673723.8529 - accuracy: 4.3860e-06
Epoch 6/15
228000/228000 [==============================] - 25s 111us/step - loss: 669038.8826 - accuracy: 4.3860e-06
Epoch 7/15
228000/228000 [==============================] - 25s 110us/step - loss: 665793.7310 - accuracy: 4.3860e-06
Epoch 8/15
228000/228000 [==============================] - 25s 111us/step - loss: 662759.0467 - accuracy: 4.3860e-06
Epoch 9/15
228000/228000 [==============================

In [18]:
def full_display(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(df)

habitaciones                                     False
garages                                          False
banos                                            False
metroscubiertos                                  False
metrostotales                                    False
fecha                                            False
gimnasio                                         False
usosmultiples                                    False
piscina                                          False
escuelascercanas                                 False
centroscomercialescercanos                       False
similares_count                                  False
porcentaje_metros                                False
diferencia_metros                                False
escomercial                                      False
promedio_metros_tipo_propiedad                   False
promedio_metros_cub_tipo_propiedad               False
tipo_propiedad_compartida                        False
prop_frecu

In [43]:

def normalizar_df(df, features, min_max=None):
    if min_max is None:
        min_max = MinMaxScaler()
        df[features] = pd.DataFrame(min_max.fit_transform(df[features]), columns=features)
        return df, min_max
    else:
        df[features] = pd.DataFrame(min_max.transform(df[features]), columns=features)
        return df, min_max
    

def llenar_nulls(df):
    df['metrostotales'] = df['metrostotales'].fillna(df['metroscubiertos'])
    df['metroscubiertos'] = df['metrostotales'].fillna(df['metrostotales'])
    
    df['habitaciones'] = df['habitaciones'].fillna(df['habitaciones'].mean())
    df['garages'] = df['garages'].fillna(df['garages'].mean())
    df['banos'] = df['banos'].fillna(df['banos'].mean())
    
    df['fecha'] = pd.to_datetime(df['fecha']).astype(int)


llenar_nulls(df_train_f)
llenar_nulls(df_test_f)


features = ['habitaciones', 'garages', 'banos', 'fecha',
       'metroscubiertos', 'metrostotales', 'provincia',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']

features_test = ['similares_count',
       'porcentaje_metros', 'diferencia_metros', 'intervalo_metros_totales',
       'intervalo_metros_cubiertos', 'escomercial',
       'promedio_metros_tipo_propiedad', 'promedio_metros_cub_tipo_propiedad',
       'tipo_propiedad_compartida', 'prop_frecuente', 'zona', 'top_provincia',
       'es_ciudad_centrica', 'promedio_metros_totales_provincia',
       'promedio_metros_cubiertos_provincia', 'anio', 'mes', 'dia',
       'trimestre', 'escualas_centros_cercanos', 'delincuencia', 'turismo',
       'es_antigua', 'cantidad_inquilinos', 'metros_totales_normalizados',
       'metros_cubiertos_normalizados', 'promedio_precio_provincia',
       'promedio_precio_ciudad', 'promedio_precio_ciudad_gen',
       'varianza_precio_ciudad', 'count_ciudad', 'promedio_id_zona',
       'promedio_id_zona_gen', 'varianza_id_zona', 'count_id_zona',
       'promedio_precio_tipo_propiedad',
       'promedio_precio_tipo_propiedad_ciudad',
       'promedio_precio_tipo_propiedad_ciudad_gen', 'count_tipo_propiedad',
       'count_tipo_propiedad_ciudad', 'promedio_por_mes', 'varianza_por_mes',
       'promedio_precio_habitaciones',
       'promedio_precio_habitaciones_banos_garages',
       'promedio_precio_banos_garages', 'promedio_precio_hbg_tipo_propiedad',
       'promedio_precio_hbg_tipo_propiedad_provincia',
       'promedio_precio_booleanos', 'puntaje']

features_cat = ['provincia', 'tipodepropiedad', 'intervalo_metros_totales', 'intervalo_metros_cubiertos',
               'zona', 'escualas_centros_cercanos']

features += features_test + features_cat


features_a_normalizar = [f for f in features if f not in features_cat + ['precio']]

df_train_g = utils.filtrar_features(df_train_f, features, 'precio')
df_train_n = pd.get_dummies(df_train_g, columns=features_cat)
df_train_n, min_max = normalizar_df(df_train_n, features_a_normalizar)

df_test_z = pd.concat([df_test_f, df_train_f.drop('precio', axis=1)], ignore_index=True, sort=False)
df_test_g = utils.filtrar_features(df_test_z, features)
df_test_n = pd.get_dummies(df_test_g, columns=features_cat).iloc[0:60000]
df_test_n, min_max = normalizar_df(df_test_n, features_a_normalizar, min_max)


In [46]:
def full_display(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(df)


In [49]:
full_display(df_test_n.describe())

,habitaciones,garages,banos,metroscubiertos,metrostotales,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,similares_count,porcentaje_metros,diferencia_metros,escomercial,promedio_metros_tipo_propiedad,promedio_metros_cub_tipo_propiedad,tipo_propiedad_compartida,prop_frecuente,top_provincia,es_ciudad_centrica,promedio_metros_totales_provincia,promedio_metros_cubiertos_provincia,anio,mes,dia,trimestre,delincuencia,turismo,es_antigua,cantidad_inquilinos,metros_totales_normalizados,metros_cubiertos_normalizados,promedio_precio_provincia,promedio_precio_ciudad,promedio_precio_ciudad_gen,varianza_precio_ciudad,count_ciudad,promedio_id_zona,promedio_id_zona_gen,varianza_id_zona,count_id_zona,promedio_precio_tipo_propiedad,promedio_precio_tipo_propiedad_ciudad,promedio_precio_tipo_propiedad_ciudad_gen,count_tipo_propiedad,count_tipo_propiedad_ciudad,promedio_por_mes,varianza_por_mes,promedio_precio_habitaciones,promedio_precio_habitaciones_banos_garages,promedio_precio_banos_garages,promedio_precio_hbg_tipo_propiedad,promedio_precio_hbg_tipo_propiedad_provincia,promedio_precio_booleanos,puntaje,provincia_Aguascalientes,provincia_Baja California Norte,provincia_Baja California Sur,provincia_Campeche,provincia_Chiapas,provincia_Chihuahua,provincia_Coahuila,provincia_Colima,provincia_Distrito Federal,provincia_Durango,provincia_Edo. de México,provincia_Guanajuato,provincia_Guerrero,provincia_Hidalgo,provincia_Jalisco,provincia_Michoacán,provincia_Morelos,provincia_Nayarit,provincia_Nuevo León,provincia_Oaxaca,provincia_Puebla,provincia_Querétaro,provincia_Quintana Roo,provincia_San luis Potosí,provincia_Sinaloa,provincia_Sonora,provincia_Tabasco,provincia_Tamaulipas,provincia_Tlaxcala,provincia_Veracruz,provincia_Yucatán,provincia_Zacatecas,provincia_otro,provincia_Aguascalientes,provincia_Baja California Norte,provincia_Baja California Sur,provincia_Campeche,provincia_Chiapas,provincia_Chihuahua,provincia_Coahuila,provincia_Colima,provincia_Distrito Federal,provincia_Durango,provincia_Edo. de México,provincia_Guanajuato,provincia_Guerrero,provincia_Hidalgo,provincia_Jalisco,provincia_Michoacán,provincia_Morelos,provincia_Nayarit,provincia_Nuevo León,provincia_Oaxaca,provincia_Puebla,provincia_Querétaro,provincia_Quintana Roo,provincia_San luis Potosí,provincia_Sinaloa,provincia_Sonora,provincia_Tabasco,provincia_Tamaulipas,provincia_Tlaxcala,provincia_Veracruz,provincia_Yucatán,provincia_Zacatecas,provincia_otro,tipodepropiedad_Apartamento,tipodepropiedad_Bodega comercial,tipodepropiedad_Casa,tipodepropiedad_Casa en condominio,tipodepropiedad_Casa uso de suelo,tipodepropiedad_Departamento Compartido,tipodepropiedad_Duplex,tipodepropiedad_Edificio,tipodepropiedad_Garage,tipodepropiedad_Hospedaje,tipodepropiedad_Huerta,tipodepropiedad_Inmuebles productivos urbanos,tipodepropiedad_Local Comercial,tipodepropiedad_Local en centro comercial,tipodepropiedad_Lote,tipodepropiedad_Nave industrial,tipodepropiedad_Oficina comercial,tipodepropiedad_Otros,tipodepropiedad_Quinta Vacacional,tipodepropiedad_Rancho,tipodepropiedad_Terreno,tipodepropiedad_Terreno comercial,tipodepropiedad_Terreno industrial,tipodepropiedad_Villa,tipodepropiedad_otro,intervalo_metros_totales_Grupo1,intervalo_metros_totales_Grupo2,intervalo_metros_totales_Grupo3,intervalo_metros_totales_Grupo4,intervalo_metros_totales_Grupo5,intervalo_metros_totales_Grupo1,intervalo_metros_totales_Grupo2,intervalo_metros_totales_Grupo3,intervalo_metros_totales_Grupo4,intervalo_metros_totales_Grupo5,intervalo_metros_cubiertos_Grupo1,intervalo_metros_cubiertos_Grupo2,intervalo_metros_cubiertos_Grupo3,intervalo_metros_cubiertos_Grupo4,intervalo_metros_cubiertos_Grupo5,intervalo_metros_cubiertos_Grupo1,intervalo_metros_cubiertos_Grupo2,intervalo_metros_cubiertos_Grupo3,intervalo_metros_cubiertos_Grupo4,intervalo_metros_cubiertos_Grupo5,zona_Centro,zona_Norte,zona_Sur,zona_Centro,zona_Norte,zona_Sur,escualas_centros_cercanos_Ambos,escualas_centros_cercanos_Centro

In [34]:
y_pred_eval = model.predict(df_test_n)
df_test_f['target'] = y_pred_eval

# df_test_f = utils.pesificar_df(df_test_f, 'target', 'target')
df_test_f[['id', 'target']].to_csv('respuesta2-keras.csv', index=False)